# Adversarial Example Generation for Images

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pdb
import pandas as pd
import numpy as np

from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
plt.rcParams["axes.grid"] = False
%matplotlib inline

In [ ]:
import torch
import torch.tensor as T
from torch import nn, optim
from torch.nn import functional as F

from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torchsummary import summary

from sklearn.metrics import accuracy_score
from skorch import NeuralNetClassifier
from skorch.callbacks import Checkpoint, Freezer, LRScheduler, EarlyStopping, Unfreezer
from skorch.dataset import CVSplit

print(f"GPU present: {torch.cuda.is_available()}")

In [ ]:
from advertorch.attacks import LinfPGDAttack

In [ ]:
img_size=(150,150)
data_stats = dict(mean=T([0.4302, 0.4575, 0.4539]), std=T([0.2361, 0.2347, 0.2433]))
data_path = Path('./data')

## Functions

In [ ]:
# def for_disp_pil(img): return transforms.ToPILImage()(img.mul(data_stats['std'][:, None, None]).add(data_stats['mean'][:, None, None]))

def for_disp(x):
  img = x.cpu()
  img = img.mul(data_stats['std'][:,None,None]).add(data_stats['mean'][:,None,None])
  img = img.numpy()
  img = img.swapaxes(1,2)
  img = img.swapaxes(2,3)
  
  return img

def get_stats(loader):
  mean,std = 0.0,0.0
  nb_samples = 0
  for imgs, _ in loader:
    batch = imgs.size(0)
    imgs = imgs.view(batch, imgs.size(1), -1)
    mean += imgs.mean(2).sum(0)
    std += imgs.std(2).sum(0)
    nb_samples += batch

  return mean/nb_samples, std/nb_samples

def sigmoid(x):
  return 1/(1 + np.exp(-x))

## EDA Data

In [ ]:
imgs,labels = [],[]
n_imgs = 5

for folder in (data_path/'train').iterdir():
  label = folder.name  
  for img_f in list(folder.glob('*.jpg'))[:n_imgs]:
    with Image.open(img_f) as f:
      imgs.append(np.array(f))
    labels.append(label)

n_classes = len(np.unique(labels))

In [ ]:
fig = plt.figure(figsize=(15, 15))

for i, img in enumerate(imgs):
  ax = fig.add_subplot(n_classes, n_imgs, i+1)
  ax.imshow(img)
  ax.set_title(labels[i], color='r')
  ax.set_xticks([])
  ax.set_yticks([])
    
plt.show()

In [ ]:
train_tfms = transforms.Compose(
  [
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(**data_stats),    
  ]
)

pred_tfms = transforms.Compose(
  [
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(**data_stats)
  ]
)

In [ ]:
ds = ImageFolder(data_path/'train', transform=train_tfms)

train_pct = 0.85
n_train = np.int(len(ds) * train_pct)
n_val = len(ds) - n_train

train_ds,val_ds = random_split(ds, [n_train, n_val])
train_ds,val_ds = train_ds.dataset,val_ds.dataset

In [ ]:
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
train_itr = iter(train_dl)

val_dl = DataLoader(val_ds, batch_size=32)
val_itr = iter(val_dl)

test_ds = ImageFolder(data_path/'test', transform=pred_tfms)
test_dl = DataLoader(test_ds, batch_size=32)

In [ ]:
imgs, labels = next(train_itr)

In [ ]:
idx = np.random.randint(len(imgs))

print(train_ds.classes[labels[idx].item()])
img = for_disp(imgs[idx])
img

In [ ]:
clf = IntelImageClassifier().cuda()
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(clf.parameters(), lr=0.01)

In [ ]:
imgs, labels = next(train_itr)
imgs = imgs.cuda()
labels = labels.cuda()

pred = clf(imgs)
loss_fn(pred, labels)

## Build Model

### Setup

In [ ]:
class UnlabeledTestSet(Dataset):
  def __init__(self, data_path, tfms=transforms.ToTensor()):
    self.imgs = list((data_path/'unlabeled_test').glob('*.jpg'))
    self.tfms = tfms
    
  def __len__(self): return len(self.imgs)
  
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
      
    with open(self.imgs[idx], 'rb') as f:
      img = Image.open(f).convert('RGB')
      
    img = self.tfms(img)
    return img, T(-1) # return dummy label

In [ ]:
tfms = {}    
tfms['train'] = transforms.Compose([
  transforms.Resize((150, 150)),
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.ToTensor(),
  transforms.Normalize(**data_stats)
])
tfms['pred'] = transforms.Compose([
  transforms.Resize((150, 150)),
  transforms.ToTensor(),
  transforms.Normalize(**data_stats)
])

train_ds = ImageFolder(data_path/'train', transform=tfms['train'])
test_ds = ImageFolder(data_path/'test', transform=tfms['pred'])
unlabeled_ds = UnlabeledTestSet(data_path, tfms['pred'])

In [ ]:
class PretrainedClf(nn.Module):
  def __init__(self, arch):
    super(PretrainedClf, self).__init__()      
    if arch == 'vgg16':
      self.model = models.vgg19_bn(pretrained=True) # load pretrained model    
      # replace last layer with custom layer
      classifier = nn.Sequential(
        nn.Linear(in_features=25088, out_features=4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=4096, out_features=6) # 6 classes
      )
      self.model.classifier = classifier
    elif arch  == 'resnet34':
      self.model = models.resnet34(pretrained=True)
      self.model.fc = nn.Linear(self.model.fc.in_features, 6)
    elif arch  == 'resnet50':
      self.model = models.resnet50(pretrained=True)
      self.model.fc = nn.Linear(self.model.fc.in_features, 6)      
    else:
      raise ValueError("Arch must be one of vgg16, resnet34 and resnet50")
    
  def forward(self, x): return self.model(x)

In [ ]:
# train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, drop_last=True)
# loss_fn = nn.CrossEntropyLoss()
# itr = iter(train_dl)

# summary(model.cuda(), input_size=(3,150,150))

# imgs, labels = itr.next()
# pred = model(imgs)
# loss = loss_fn(pred, labels)
# print(loss)

In [ ]:
arch = 'vgg16'
model = PretrainedClf(arch)

if arch == 'vgg16': layer_name = 'model.classifier'
elif arch in ['resnet34', 'resnet50']: layer_name = 'model.fc'

In [ ]:
lrscheduler = LRScheduler(policy='StepLR', step_size=7, gamma=0.1)
chkpt = Checkpoint(dirname=Path(f'./workdir/{arch}'), monitor='valid_acc_best')
freezer = Freezer(lambda x: not x.startswith(layer_name))
unfreezer = Unfreezer(lambda x: not x.startswith(layer_name))

callbacks = [lrscheduler, chkpt, freezer, EarlyStopping()]

In [ ]:
net = NeuralNetClassifier(
  model,
  criterion=nn.CrossEntropyLoss,
  lr=0.001,
  batch_size=128,
  max_epochs=25,
  optimizer=optim.Adam,
  iterator_train__shuffle=True,
  iterator_train__drop_last=True,
  iterator_train__num_workers=4,
  iterator_valid__shuffle=False,  
  train_split=CVSplit(cv=0.15),
  callbacks=callbacks,
  device='cuda:0'
)

### Training

In [ ]:
net.fit(train_ds, y=None);

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
n_epochs = len(net.history)
sns.lineplot(range(n_epochs), net.history[:, 'train_loss'])
sns.lineplot(range(n_epochs), net.history[:, 'valid_loss'])
ax.legend(['train_loss', 'valid_loss'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')

In [ ]:
net.callbacks.remove(freezer)
net.callbacks.append(unfreezer)
net.callbacks

In [ ]:
net.partial_fit(train_ds, y=None);

### Inference

In [ ]:
net.initialize();
net.load_params(checkpoint=chkpt)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
n_epochs = len(net.history)
sns.lineplot(range(n_epochs), net.history[:, 'train_loss'])
sns.lineplot(range(n_epochs), net.history[:, 'valid_loss'])
ax.legend(['train_loss', 'valid_loss'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')

In [ ]:
pred = net.predict(test_ds)
test_dl = DataLoader(test_ds, batch_size=len(test_ds))
_, y_true = iter(test_dl).next()
y_true = y_true.numpy()

print(f"Accuracy of labeled test set= {accuracy_score(y_true, pred)*100:0.2f}%")

In [ ]:
unlabeled_ds = UnlabeledTestSet(data_path, tfms['pred'])
unlabeled_dl = DataLoader(unlabeled_ds, batch_size=len(unlabeled_ds))
itr = iter(unlabeled_dl)

imgs, _ = itr.next()

In [ ]:
idx = np.random.randint(len(imgs))
print(f"Predicted Label: {train_ds.classes[net.predict(imgs[idx].unsqueeze(dim=0)).item()]}")
for_disp(imgs[idx])

## Adversarial Perturbations

### Setup

In [ ]:
net.initialize();
net.load_params(checkpoint=chkpt)

trained_model = net.module.eval()
trained_model = trained_model.eval()
trained_model = trained_model.cuda()

unt_adversary = LinfPGDAttack(trained_model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.15, nb_iter=40, eps_iter=0.01, rand_init=True, clip_min=0.0, clip_max=1.0, targeted=False)
t_adversary = LinfPGDAttack(trained_model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.15, nb_iter=40, eps_iter=0.01, rand_init=True, clip_min=0.0, clip_max=1.0, targeted=True)

### Untargeted

In [ ]:
bs = 128
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)
test_itr = iter(test_dl)

In [ ]:
%%time
x_clean, y_true = test_itr.next()
x_clean,y_true = x_clean.cuda(),y_true.cuda()
x_unt_adv = unt_adversary.perturb(x_clean, y_true)
y_pred,y_unt_pred = net.predict(x_clean), net.predict(x_unt_adv)
y_prob,y_unt_prob = sigmoid(net.predict_proba(x_clean)), sigmoid(net.predict_proba(x_unt_adv))
y_true = y_true.cpu().numpy()

In [ ]:
print(f"Accuracy of {bs} clean examples= {accuracy_score(y_true, y_pred)*100:0.2f}%")
print(f"Accuracy of {bs} adversarial examples= {accuracy_score(y_true, y_unt_pred)*100:0.2f}%")

In [ ]:
imgs,imgs_adv = for_disp(x_clean),for_disp(x_unt_adv)
imgs_diff = np.abs(imgs-imgs_adv)

In [ ]:
np.max(y_prob[idxs[i]])

In [ ]:
samples = min(5, len(imgs))
idxs = np.random.choice(len(imgs), (samples), replace=False)

fig,axs = plt.subplots(nrows=4, ncols=3, figsize=(20, 16))
for i, ax in enumerate(axs[None:]):
  ax[0].imshow(imgs[idxs[i]])
  ax[0].set_title(f"True: {train_ds.classes[y_true[idxs[i]]]}, Pred: {train_ds.classes[y_pred[idxs[i]]]}, Conf: {np.max(y_prob[idxs[i]])*100:0.2f}", color='b')
  ax[0].axis('off')
  ax[0].axis('tight')
  
  ax[1].imshow(imgs_adv[idxs[i]])
  ax[1].set_title(f"Untargeted, Pred: {train_ds.classes[y_unt_pred[idxs[i]]]}, Conf: {np.max(y_unt_prob[idxs[i]])*100:0.2f}", color='r')
  ax[1].axis('off')
  ax[1].axis('tight')
  
  ax[2].imshow(imgs_diff[idxs[i]])
  ax[2].set_title("Image difference", color='g')
  ax[2].axis('off')
  ax[2].axis('tight')  

### Targeted

In [ ]:
test_ds.classes

In [ ]:
source,target = 'forest','street'
source_idxs = [i for i in range(len(test_ds.targets)) if test_ds.targets[i] == test_ds.class_to_idx[source]]
test_ds.samples = [test_ds.samples[idx] for idx in source_idxs]
test_ds.targets = [test_ds.targets[idx] for idx in source_idxs]

In [ ]:
bs = 128
target_labels = T([test_ds.class_to_idx[target]] * bs).cuda()
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)
test_itr = iter(test_dl)

In [ ]:
%%time
x_clean, y_true = test_itr.next()
x_clean,y_true = x_clean.cuda(),y_true.cuda()
x_t_adv = t_adversary.perturb(x_clean, target_labels)
y_pred,y_t_pred = net.predict(x_clean), net.predict(x_t_adv)
y_true = y_true.cpu().numpy()

In [ ]:
y_t_prob = net.predict_proba(x_t_adv)

In [ ]:
y_t_prob = sigmoid(y_t_prob)

In [ ]:
print(f"Accuracy of {bs} clean examples= {accuracy_score(y_true, y_pred)*100:0.2f}%")
print(f"Accuracy of {bs} adversarial examples= {accuracy_score(y_true, y_t_pred)*100:0.2f}%")

In [ ]:
imgs,imgs_adv = for_disp(x_clean),for_disp(x_t_adv)
imgs_diff = np.abs(imgs-imgs_adv)

In [ ]:
samples = min(5, len(imgs))
idxs = np.random.choice(len(imgs), (samples), replace=False)

fig,axs = plt.subplots(nrows=4, ncols=3, figsize=(20, 15))
for i, ax in enumerate(axs[None:]):
  ax[0].imshow(imgs[idxs[i]])
  ax[0].set_title(f"Clean Example\nPrediction: {train_ds.classes[y_pred[idxs[i]]]}", color='r')
  ax[0].axis('off')
  ax[0].axis('tight')
  
  ax[1].imshow(imgs_adv[idxs[i]])
  ax[1].set_title(f"Targeted Aversarial {source}-->{target}\nPrediction: {train_ds.classes[y_t_pred[idxs[i]]]}", color='r')
  ax[1].axis('off')
  ax[1].axis('tight')
  
  ax[2].imshow(imgs_diff[idxs[i]])
  ax[2].set_title("Image difference")
  ax[2].axis('off')
  ax[2].axis('tight')  